# make_geojson.ipynb

### 概要

宇都宮市の[オープンソースライブラリ](https://data.city.utsunomiya.tochigi.jp/)から、[防災データ](https://catalog.city.utsunomiya.tochigi.jp/dataset/shiteikinkyuuhinanbashoichiran)を用いて公開するgeojsonファイルを作成します

In [1]:
# 指定緊急避難場所一覧（令和３年８月現在）
csv_data_path='https://catalog.city.utsunomiya.tochigi.jp/dataset/4d41b0e3-b48a-4079-8896-7de6f6f1a850/resource/ab7c2ca6-9ce3-409a-947b-4744fcfec7c6/download/092011_evacuation_space.csv'

In [3]:
import pandas as pd # データフレーム
import requests # csvへのリクエスト
import io # ファイル名の管理

#### csvデータの読み取り

In [7]:
r = requests.get(csv_data_path)
df = pd.read_csv(io.BytesIO(r.content),sep=",",encoding='shift-jis',index_col=0)

In [8]:
df

,名称,名称_カナ,住所,方書,緯度,経度,標高,電話番号,内線番号,市区町村コード,...,災害種別_地震,災害種別_津波,災害種別_大規模な火事,災害種別_内水氾濫,災害種別_火山現象,指定避難所との重複,想定収容人数,対象となる町会・自治会,URL,備考
NO,,,,,,,,,,,,,,,,,,,,,
1,中央小学校,チュウオウショウガッコウ,栃木県宇都宮市中央本町1-29,NaN,36.559260,139.884772,NaN,028-635-3043,NaN,92011,...,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
2,東小学校,ヒガシショウガッコウ,栃木県宇都宮市東塙田1-6-14,NaN,36.565214,139.891120,NaN,028-622-2282,NaN,92011,...,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
3,西小学校,ニシショウガッコウ,栃木県宇都宮市西1-2-13,NaN,36.559704,139.878273,NaN,028-634-6129,NaN,92011,...,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
4,簗瀬小学校,ヤナセショウガッコウ,栃木県宇都宮市南大通り2-6-6,NaN,36.554581,139.893488,NaN,028-633-0363,NaN,92011,...,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
5,西原小学校,ニシハラショウガッコウ,栃木県宇都宮市西原2-5-42,NaN,36.554081,139.872789,NaN,028-634-4401,NaN,92011,...,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,北市民活動センター,キタシミンカツドウセンター,栃木県宇都宮市若草3-12-25,NaN,36.587305,139.861600,NaN,028-622-7093,NaN,92011,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
195,晃宝小学校,コウホウショウガッコウ,栃木県宇都宮市宝木本町1263-1,NaN,36.606501,139.854281,NaN,028-665-5295,NaN,92011,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
196,国本地区市民センター,クニモトチクシミンセンター,栃木県宇都宮市宝木本町1868-1,NaN,36.615071,139.839651,NaN,028-665-1041,NaN,92011,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


#### geojsonの作成

In [21]:
df.size

4950